In [1]:
import sys
sys.path.append("../")
import shared
from Utils.eth_utils import *
import json
import requests

In [2]:
import pandas as pd

In [31]:
pools = pd.read_csv("../data/polygon_pools.csv")

In [32]:
pools = pools.loc[(pools.token0==shared.WMATIC) | (pools.token1==shared.WMATIC)]

In [33]:
pools.

,pair,token0,token1,fees,block_number,timestamp
3,0xFE530931dA161232Ec76A7c3bEA7D36cF3811A0d,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063,3000,22771167,1640051661
6,0xF4BE3782A86A65E053EDDBBbF19A06E445B8dDf6,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,0xf50D05A1402d0adAfA880D36050736f9f6ee7dee,10000,22797886,1640109456
13,0x9B08288C3Be4F62bbf8d1C20Ac9C5e6f9467d8B7,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,0xc2132D05D31c914a87C6611C10748AEb04B58e8F,500,22799345,1640112714
14,0x781067Ef296E5C4A4203F81C593274824b7C185d,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,0xc2132D05D31c914a87C6611C10748AEb04B58e8F,3000,22799346,1640112718
15,0x9a72FC3Fb9E99087d2eAE500355E7902C763f9B3,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,0xc2132D05D31c914a87C6611C10748AEb04B58e8F,10000,22799346,1640112718
...,...,...,...,...,...,...
2480,0x0f4A25618CE9009ec256E4Cc14D8C88F804e7190,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,0x4BcB3602BeF5DE409c26CFDCC72eD41792510e8e,3000,27466100,1650683502
2481,0xC7749265d0543Eb8683a3885d8940595F623b9D6,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,0x1943745CDD660FF432e88Aef09990e35f1E6AE1f,3000,27466100,1650683502
2482,0x3727EBf05E60F61065959bE2FFDd234B3b1C130a,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,0xc57a2940D60e7c952126FB32A86785100d52BC4F,3000,27466109,1650683524
2483,0x9275a6874aC015b97C7805484F6A72934dD72cc9,0x01C8Ef38EE968fA3B54Efd71bFD41aeb686338c8,0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270,3000,27466110,1650683528


In [10]:
pools = pools.values.tolist()

In [12]:
pools

[['0x6baD0f9a89Ca403bb91d253D385CeC1A2b6eca97',
  '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619',
  '0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063',
  3000,
  22757913,
  1640022872],
 ['0x0e44cEb592AcFC5D3F09D996302eB4C499ff8c10',
  '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174',
  '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619',
  3000,
  22762313,
  1640032080],
 ['0x45dDa9cb7c25131DF268515131f647d726f50608',
  '0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174',
  '0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619',
  500,
  22765962,
  1640039738],
 ['0xFE530931dA161232Ec76A7c3bEA7D36cF3811A0d',
  '0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270',
  '0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063',
  3000,
  22771167,
  1640051661],
 ['0x78Df0bfFFf7bc653b9a8b14Cd1C0E740d1407F42',
  '0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063',
  '0xf50D05A1402d0adAfA880D36050736f9f6ee7dee',
  3000,
  22797812,
  1640109304],
 ['0x0e226447ae42F91b338B25c7DD23D20E337F78a3',
  '0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063',
  '0xf50D0

In [13]:
import networkx as nx

In [14]:
G = nx.Graph()

In [15]:
for pool in pools:
    token0 =pool[1]
    token1 = pool[2]
    G.add_edge(token0,token1)

In [16]:
nodes_degree = [(x,G.degree(x)) for x in G.nodes()]

In [22]:
wmatic = "0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270"

In [21]:
sorted(nodes_degree, key=lambda x: x[1],reverse=True)

[('0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270', 934),
 ('0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174', 366),
 ('0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619', 207),
 ('0xc2132D05D31c914a87C6611C10748AEb04B58e8F', 139),
 ('0x8f3Cf7ad23Cd3CaDbD9735AFf958023239c6A063', 85),
 ('0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6', 32),
 ('0x550D11714ad6DdEe6916B18fE67285c261903f24', 28),
 ('0x7Ecb5699D8E0a6572E549Dc86dDe5A785B8c29BC', 19),
 ('0x7D645CBbCAdE2A130bF1bf0528b8541d32D3f8Cf', 18),
 ('0xb33EaAd8d922B1083446DC23f610c2567fB5180f', 16),
 ('0x67B78D31A40dACC5aCf7648C2Ad88bDBF6EeEc1C', 16),
 ('0xD6DF932A45C0f255f85145f286eA0b292B21C90B', 16),
 ('0xA1c57f48F0Deb89f569dFbE6E2B7f46D33606fD4', 13),
 ('0xa3Fa99A148fA48D14Ed51d610c367C61876997F1', 13),
 ('0xA9B8e4BE7e4Fc1B1d28B396af7c4CBa37979bC0F', 13),
 ('0x1fe40DAe6923530B392d5A86A3E7006218CBDbd4', 13),
 ('0x20ce796B55Aaad56BD4205EED6011B2A19bC9F7E', 13),
 ('0x172370d5Cd63279eFa6d502DAB29171933a610AF', 12),
 ('0x6AE7Dfc73E0dDE2aa99ac063DcF7e8A632651

In [28]:
tokens = [x for x in G.neighbors(wmatic)]

In [29]:
tokens

['...']